Preparing the data

In [298]:
import numpy as np
import pandas as pd
from scipy.stats import norm

data = pd.read_csv("data425288.csv")

T = 25
N = data.ID.max()

y = np.log(data.Sales.values.reshape((N,T)))

X = np.ones((T,2))
X[:,1] = np.log(data.Price[:T])

Log-likelihood evaluation

In [299]:
def LogL(theta, pi, y, X):
    mu = np.dot(X,theta.T) #(25, 3)
    mu = np.repeat(mu[np.newaxis, :, :], N, axis=0)  #(500,25,K)
    y = np.repeat(y[:, :, np.newaxis], K, axis=2)  #(500,25,K)
    
    #pdfs
    probs = norm.pdf(y,mu,1) #(500,25,K)
    
    #prod^T probs
    segments = np.prod(probs, axis=1) #(500,K)
    
    #sum^K pi_c prod^T probs
    combined = np.dot(segments, pi) #(500,)
    
    #sum^N log sum^K pi_c prod^T probs
    LogL = np.log(combined).sum() #(1,)
    return LogL

Expectation step

In [454]:
def EStep(theta,pi,y,X):
    mu = np.dot(X,theta.T) #(25, K)
    mu = np.repeat(mu[np.newaxis, :, :], N, axis=0)  #(500,25,K)
    y = np.repeat(y[:, :, np.newaxis], K, axis=2)  #(500,25,K)

    #pdfs
    probs = norm.pdf(y,mu,1) #(500,25,K)
    
    #prod^T probs
    segments = np.log(probs).sum(axis=1)
#     segments = np.prod(probs, axis=1) #(500,K)
    print(segments.shape)
    print(segments)
    
    # prod^T probs times diagonal of pi
#     numerators = np.dot(segments, np.diag(pi))
    numerators = np.exp(segments + np.log(pi) - segments.min())

    print(numerators)
    
    
    #divide numeratats by denominators (= sum of row)
    W = numerators / numerators.sum(axis=1, keepdims=True)
    
    return W

W = EStep(theta, pi, y, X)
display(W.shape)
display(W)

(500, 3)
[[-1375.0105342  -1375.0105342  -1375.0105342 ]
 [-1672.68915241 -1672.68915241 -1672.68915241]
 [-1363.24234988 -1363.24234988 -1363.24234988]
 ...
 [-1613.11765141 -1613.11765142 -1613.11765142]
 [-1315.51633398 -1315.51633398 -1315.51633398]
 [-1303.38790321 -1303.38790321 -1303.38790321]]
[[7.39371533e+170 1.10905730e+171 1.84842883e+171]
 [3.87865773e+041 5.81798660e+041 9.69664433e+041]
 [9.54377603e+175 1.43156640e+176 2.38594401e+176]
 ...
 [2.88572949e+067 4.32859423e+067 7.21432372e+067]
 [5.09173257e+196 7.63759886e+196 1.27293314e+197]
 [9.42271690e+201 1.41340753e+202 2.35567922e+202]]


(500, 3)

array([[0.2, 0.3, 0.5],
       [0.2, 0.3, 0.5],
       [0.2, 0.3, 0.5],
       ...,
       [0.2, 0.3, 0.5],
       [0.2, 0.3, 0.5],
       [0.2, 0.3, 0.5]])

Maximization step

In [453]:
def MStep(W,y,X):
    # 1/N sum^N w_ic
    pi = W.mean(axis=0) #(K,)
    
    # sum^N w_ic sum^T log S_it
    numerator = np.dot(y.sum(axis=1), W) #(K,)
    
    #  / T * sum^N w_ic
    alpha = numerator / (T * W.sum(axis=0))
    
    # / sum^N w_ic * sum^T log p_t (scalar)
    beta =  numerator /  (W.sum(axis=0) * X[:,1].sum())
    
    # [a', b']
    theta = np.stack((alpha, beta), axis=-1)
    
    return theta, pi
    
theta, pi = MStep(W,y,X)

# theta[:,1] = -theta[:,1] 
# theta = theta /12
display(pi)    

display(theta)    
display(theta.shape)
# print(X)




array([0.2, 0.3, 0.5])

array([[9.32891466, 9.42045641],
       [9.32891466, 9.42045641],
       [9.32891466, 9.42045641]])

(3, 2)

EM algorithm

In [450]:
#test params
K = 3

# np.random.seed(1234)
# alpha = np.random.rand(K) * 3 + 3
# beta = np.random.rand(K) - 1
# theta = np.stack((alpha, beta), axis=-1)

W = np.ones((N,K))
W[:,0] = 0.2
W[:,1] = 0.3
W[:,2] = 0.5

    
print(W)


[[0.2 0.3 0.5]
 [0.2 0.3 0.5]
 [0.2 0.3 0.5]
 ...
 [0.2 0.3 0.5]
 [0.2 0.3 0.5]
 [0.2 0.3 0.5]]


In [404]:
likelihood = LogL(theta, pi, y, X)
display(likelihood.shape)
display(likelihood)

C:\Users\Tim\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


()

-inf

In [ ]:
def EM(K,y,X):
    
    

Estimation implementation

In [ ]:
def Estimate(K, X=X, y=y, seed=1234):